In [12]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import pandas as pd

#Bước 1: Đọc file dữ liệu gốc từ Excel
file_path = "Dataset/DL_ma_hoa_goc.xlsx"
df = pd.read_excel(file_path)

# Bước 2: Loại bỏ các hàng chứa giá trị thiếu (NaN) và các hàng trùng lặp hoàn toàn
df_cleaned = df.dropna().drop_duplicates()

# Bước 3: Ép kiểu cột GPA và NumberTL về kiểu số; nếu lỗi chuyển thành NaN
df_cleaned['GPA'] = pd.to_numeric(df_cleaned['GPA'], errors='coerce')
df_cleaned['NumberTL'] = pd.to_numeric(df_cleaned['NumberTL'], errors='coerce')


# Bước 4: Giữ lại các dòng có GPA hợp lệ trong khoảng [0, 4]
df_cleaned = df_cleaned[(df_cleaned['GPA'] >= 0) & (df_cleaned['GPA'] <= 4)]

# Bước 5: Giữ lại các sinh viên có NumberTL tăng dần theo học kỳ
def is_tl_monotonic(group):
    return group.sort_values('Học kì')['NumberTL'].is_monotonic_increasing
df_cleaned = df_cleaned.groupby('MSSV').filter(is_tl_monotonic)

# Bước 6: Giữ lại các sinh viên có học kỳ liên tiếp từ 1 đến N (không bị thiếu kỳ)
def is_sequential(group):
    return (group['Học kì'].sort_values().values == list(range(1, len(group) + 1))).all()
df_cleaned = df_cleaned.groupby('MSSV').filter(is_sequential)

# Bước 7: Tạo bảng tín chỉ qua 
df_cleaned['TC qua'] = df_cleaned.groupby('MSSV')['NumberTL'].diff().fillna(df_cleaned['NumberTL'])


def is_monotonic(group):
    return group['NumberTL'].is_monotonic_increasing

df_cleaned = df_cleaned.groupby('MSSV').filter(is_monotonic)

# Bước 8: Giữ lại cột cần thiết và đưa EncryptedID lên đầu
final_df =df_cleaned[['MSSV', 'Học kì', 'GPA','TC qua', 'NumberTL']].copy()


# Bước 8: Lưu dữ liệu đã làm sạch ra file Excel
output_path = "Data_clean/Data_GPA_CPA_1.xlsx"
final_df.to_excel(output_path, index=False)

print("✅ Đã lưu file dữ liệu sạch tại:", output_path)


✅ Đã lưu file dữ liệu sạch tại: Data_clean/Data_GPA_CPA_1.xlsx


In [13]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import pandas as pd

# Bước 1: Đọc dữ liệu
file_path = "Dataset/ET1_K62_K63_K64_performance.csv"
df = pd.read_csv(file_path)

# Bước 2: Giữ lại các cột cần thiết
df = df[['Semester', 'GPA', 'Acc', 'EncryptedID', 'TC qua']].copy()

# Bước 3: Loại bỏ dòng thiếu và trùng
df_cleaned = df.dropna().drop_duplicates()

# Bước 4: Ép kiểu GPA và Acc
df_cleaned['GPA'] = pd.to_numeric(df_cleaned['GPA'], errors='coerce')
df_cleaned['Acc'] = pd.to_numeric(df_cleaned['Acc'], errors='coerce')

# Bước 5: Loại GPA không hợp lệ
df_cleaned = df_cleaned[(df_cleaned['GPA'] >= 0) & (df_cleaned['GPA'] <= 4)]

# Bước 6: Chuẩn hóa học kỳ: đổi 'Semester' về 1, 2, 3,... theo từng sinh viên
df_cleaned = df_cleaned.sort_values(['EncryptedID', 'Semester']).copy()
df_cleaned['Semester'] = df_cleaned.groupby('EncryptedID').cumcount() + 1

# Bước 7: Loại sinh viên có Acc không tăng dần theo học kỳ
def is_monotonic(group):
    return group['Acc'].is_monotonic_increasing

df_cleaned = df_cleaned.groupby('EncryptedID').filter(is_monotonic)

# Bước 8: Giữ lại cột cần thiết và đưa EncryptedID lên đầu
final_df = df_cleaned[['EncryptedID', 'Semester', 'GPA', 'TC qua', 'Acc']].copy()

# Bước 9: Lưu ra Excel
output_path = "Data_clean/Data_CPA_CPA_ET2.csv"
final_df.to_csv(output_path, index=False)

print("✅ Đã lưu file dữ liệu sạch tại:", output_path)

✅ Đã lưu file dữ liệu sạch tại: Data_clean/Data_CPA_CPA_ET2.csv


In [14]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import pandas as pd

# Bước 1: Đọc dữ liệu
file_path_1 = "Data_clean\Data_GPA_CPA_1.xlsx"
file_path_2 = "Data_clean\Data_CPA_CPA_ET2.csv"
df1 = pd.read_excel(file_path_1)
df2 = pd.read_csv(file_path_2)

# Bước 2: Đổi tên cột MSSV thành EncryptedID, NumberTL thành Acc, Học kỳ thành Semester trong df1
df1.rename(columns={
    'MSSV': 'EncryptedID', 
    'NumberTL': 'Acc', 
    'Học kì': 'Semester'
    }, inplace=True)

# Bước 3: Giữ lại các cột cần thiết
df1 = df1[['EncryptedID', 'Semester', 'GPA', 'TC qua', 'Acc']].copy()
df2 = df2[['EncryptedID', 'Semester', 'GPA', 'TC qua', 'Acc']].copy()

# Bước 4: Gộp hai DataFrame
final_df = pd.concat([df1, df2], ignore_index=True)

# Bước 5: Loại bỏ các dòng trùng lặp
final_df = final_df.drop_duplicates()
# Bước 6: Lưu ra file Excel
output_path = "Data_clean/Data_GPA_CPA_final.xlsx"
final_df.to_excel(output_path, index=False)
print("✅ Đã lưu file dữ liệu cuối cùng tại:", output_path)



✅ Đã lưu file dữ liệu cuối cùng tại: Data_clean/Data_GPA_CPA_final.xlsx


In [2]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import pandas as pd

# Bước 1: Đọc dữ liệu
file_path = "Data_clean/Data_GPA_CPA_final.xlsx"
file_path_EE = "Data_clean\EE2_K62_K63_K64_10_ky_clean.csv" 

df = pd.read_excel(file_path)
df_ee = pd.read_csv(file_path_EE)

df_ee.rename(columns={
    'TC_1': 'TC_qua_1', 
    'TC_2': 'TC_qua_2',
    'TC_3': 'TC_qua_3',
    'TC_4': 'TC_qua_4',
    'TC_5': 'TC_qua_5',
    'TC_6': 'TC_qua_6',
    'TC_7': 'TC_qua_7',
    'TC_8': 'TC_qua_8',
    'TC_9': 'TC_qua_9',
    'TC_10': 'TC_qua_10'
    }, inplace=True)

# Bước 2: Tách sinh viên học đúng 10 kỳ và 8 kỳ
sv_10_ky = df.groupby("EncryptedID").filter(lambda x: len(x) == 10)
sv_8_ky = df.groupby("EncryptedID").filter(lambda x: len(x) == 8)


# Bước 3: Loại sinh viên có GPA kỳ cuối bằng 0 
def remove_students_with_zero_final_gpa(data, so_ky):
    return data.groupby("EncryptedID").filter(
        lambda x: not x[x["Semester"] == so_ky]["GPA"].eq(0).any()
    )

# Bước 4: Loại sinh viên có tổng tín chỉ tích lũy cuối kỳ < ngưỡng ---
def remove_students_with_insufficient_credits(data, so_ky, tc_threshold):
    return data.groupby("EncryptedID").filter(
        lambda x: x[x["Semester"] == so_ky]["Acc"].values[0] >= tc_threshold
    )

sv_10_ky = remove_students_with_zero_final_gpa(sv_10_ky, 10)
sv_10_ky = remove_students_with_insufficient_credits(sv_10_ky, 10, 150)
sv_8_ky = remove_students_with_zero_final_gpa(sv_8_ky, 8)
sv_8_ky = remove_students_with_insufficient_credits(sv_8_ky, 8, 120)

# Bước 5: Tính Final CPA
def calculate_final_cpa(data, so_ky):
    gpa_pivot = data.pivot(index="EncryptedID", columns="Semester", values="GPA")
    gpa_pivot.columns = [f"GPA_{i}" for i in gpa_pivot.columns]

    Acc_pivot = data.pivot(index="EncryptedID", columns="Semester", values="Acc")
    Acc_pivot.columns = [f"TC_LK_{i}" for i in Acc_pivot.columns]

    tc_qua_pivot = data.pivot(index="EncryptedID", columns="Semester", values="TC qua")
    tc_qua_pivot.columns = [f"TC_qua_{i}" for i in tc_qua_pivot.columns]

    
    numerator = sum(gpa_pivot[f"GPA_{i}"] * tc_qua_pivot[f"TC_qua_{i}"] for i in range(1, so_ky + 1))
    denominator = sum(tc_qua_pivot[f"TC_qua_{i}"] for i in range(1, so_ky + 1))
    final_cpa = (numerator / denominator).round(2)

    # result = gpa_pivot.copy()
    result = pd.concat([gpa_pivot, tc_qua_pivot, Acc_pivot], axis=1)
    result["Final_CPA"] = final_cpa
    result =  result.reset_index()
    return result

# Bước 6: Tính Final CPA cho sinh viên
sv_10_ky_final = calculate_final_cpa(sv_10_ky, 10)
# sv_10_ky_final = pd.concat([sv_10_ky_final, df_ee], ignore_index=True)
# sv_10_ky_final = sv_10_ky_final.dropna().drop_duplicates()
sv_8_ky_final = calculate_final_cpa(sv_8_ky, 8) 

# Bước 7: Lưu kết quả ra file Excel
output_path_10_ky = "Data_clean/Data_GPA_CPA_final_10_ky.xlsx"
output_path_8_ky = "Data_clean/Data_GPA_CPA_final_8_ky.xlsx"
sv_10_ky_final.to_excel(output_path_10_ky, index=False)    
sv_8_ky_final.to_excel(output_path_8_ky, index=False)   


print("✅ Đã lưu file dữ liệu cuối cùng cho sinh viên 10 kỳ tại:", output_path_10_ky)
print("✅ Đã lưu file dữ liệu cuối cùng cho sinh viên 8 kỳ tại:", output_path_8_ky)


✅ Đã lưu file dữ liệu cuối cùng cho sinh viên 10 kỳ tại: Data_clean/Data_GPA_CPA_final_10_ky.xlsx
✅ Đã lưu file dữ liệu cuối cùng cho sinh viên 8 kỳ tại: Data_clean/Data_GPA_CPA_final_8_ky.xlsx
